In [1]:
import xyzpy as xyz
import numpy as np

def trig(f, x, amp, phi):

    trig_fn = getattr(np, f)
    err = 0.2 + 0.1 * np.random.randn()

    return amp * trig_fn(x - phi), err

r = xyz.Runner(trig, ['f(x)', 'ef(x)'])

The harvester is composed of this ``Runner`` and logic for merging for saving each new set of runs.

In [2]:
h = xyz.Harvester(r, data_name='trig.h5')

We perform one set of runs first:

In [3]:
combos_1 = {
    'f': ['sin', 'cos'],
    'x': np.linspace(-5, 5, 101),
    'amp': [1, 2, 3],
    'phi': [0.4, 0.8],
}

In [4]:
h.harvest_combos(combos_1)

100%|##########| 1212/1212 [00:00<00:00, 110364.44it/s]


Which we can plot:

In [5]:
h.full_ds.xyz.ilineplot(x='x', y='f(x)', z='f', col='amp', row='phi', y_err='ef(x)')

Loading BokehJS ...

Then we can define and run a completely disjoint set of runs:

In [6]:
combos_2 = {
    'f': ['sin', 'cos'],
    'x': np.linspace(-5, 5, 101),
    'amp': [4, 5],
    'phi': [1.2, 1.6],
}

In [7]:
h.harvest_combos(combos_2)

100%|##########| 808/808 [00:00<00:00, 80894.58it/s]


If we plot the current ``full_ds`` we’ll see both sets of runs are present, but there are also blank spaces:

In [8]:
h.full_ds.xyz.ilineplot(x='x', y='f(x)', z='f', col='amp', row='phi', y_err='ef(x)')

Finally let’s add ``tan`` into the mix, but only for the central combinations:

In [9]:
combos_3 = {
    'f': ['tan'],
    'x': np.linspace(-5, 5, 101),
    'amp': [2, 3, 4],
    'phi': [0.8, 1.2],
}

In [10]:
h.harvest_combos(combos_3)

100%|##########| 606/606 [00:00<00:00, 105994.50it/s]


We’ll specify some plotting limits to avoid the infinities as well:

In [11]:
h.full_ds.xyz.ilineplot(
    x='x', y='f(x)', z='f',
    col='amp', row='phi', y_err='ef(x)',
    ylims=(-4, 4),
)

All runs have also been accumulated into the on-disk:

In [12]:
ls *.h5

trig.h5


In [13]:
xyz.load_ds('trig.h5')

<xarray.Dataset>
Dimensions:  (amp: 5, f: 3, phi: 4, x: 101)
Coordinates:
  * amp      (amp) int64 1 2 3 4 5
  * f        (f) object 'cos' 'sin' 'tan'
  * phi      (phi) float64 0.4 0.8 1.2 1.6
  * x        (x) float64 -5.0 -4.9 -4.8 -4.7 -4.6 -4.5 ... 4.6 4.7 4.8 4.9 5.0
Data variables:
    ef(x)    (f, x, amp, phi) float64 0.3228 0.1255 nan nan ... nan nan nan nan
    f(x)     (f, x, amp, phi) float64 0.6347 0.8855 nan nan ... nan nan nan nan

For now just clean this up.

In [14]:
h.delete_ds()